In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
spreadsheet = pd.ExcelFile('Dataset/dataset.xlsx')
df2 = spreadsheet.parse('Sheet1')
df2.head()

State_Name District_Name  Crop_Year       Season       Crop  Area(ha)  \
0  Maharashtra    AHMEDNAGAR     1997.0  Kharif       Arhar/Tur   17600.0   
1  Maharashtra    AHMEDNAGAR     1997.0  Kharif           Bajra  274100.0   
2  Maharashtra    AHMEDNAGAR     1997.0  Kharif            Gram   40800.0   
3  Maharashtra    AHMEDNAGAR     1997.0  Kharif           Jowar     900.0   
4  Maharashtra    AHMEDNAGAR     1997.0  Kharif           Maize    4400.0   

   Production(tonnes)  Avg_Prep(mm)  Avg_temp(C)  Yield(tonnes/ha)  
0              6300.0     136.30175        25.65          0.357955  
1            152800.0     136.30175        25.65          0.557461  
2             18600.0     136.30175        25.65          0.455882  
3              1100.0     136.30175        25.65          1.222222  
4              4700.0     136.30175        25.65          1.068182

In [17]:
crops_combo=[]
crop_season=[]
for row in df2.itertuples():
    #use row[5] strip
    crop_season.append(row[5])
    #use row[4] strip
    crop_season.append(row[4])
    crops_combo.append(crop_season)
    crop_season=[]
a = set(tuple(i) for i in crops_combo)
print('length of set :',len(a))
for l in sorted(a):
    print(l[0]," ",l[1])

length of set : 41


TypeError: '<' not supported between instances of 'float' and 'str'

In [18]:
ar = []
ar2d = []

x_list=[]
z_list=[]
i=0
y_list=[]

model_list = []
for l in sorted(a):
    for row in df2.itertuples():
        if row[5].strip()==l[0] and row[4].strip()==l[1]:
            ar.append(row[8])
            x_list.append(row[8])
            ar.append(row[9])
            z_list.append(row[9])
            y_list.append(row[10])
            ar2d.append(ar)
            ar = []
    if len(ar2d) < 10:
        continue
    
    nar2d = np.array(ar2d)
    nar2d = np.nan_to_num(nar2d)
    ny = np.array(y_list)
    ny = np.nan_to_num(ny)
    
    X_train = nar2d[:-4]
    X_test = nar2d[-4:]

    y_train = ny[:-4]
    y_test = ny[-4:]

    # Fit line using all data
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    
    line_y = lr.predict(X_test)

    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_train, y_train)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    line_y_ransac = ransac.predict(X_test)
    
    model_list.append([ransac,l[0],l[1]])
    #print("ransac prediction")
    #print(line_y_ransac)
    # Compare estimated coefficients
    #print("Estimated coefficients (linear regression, RANSAC):")
    #print(lr.coef_, ransac.estimator_.coef_)

    # The mean squared error
    print("Mean squared error of linear regression: %.2f"
          % mean_squared_error(y_test, line_y))
    print("Mean squared error of ramsac regression: %.2f"
          % mean_squared_error(y_test, line_y_ransac))
    # Explained variance score: 1 is perfect prediction
    print('Variance score of linear regr: %.2f' % r2_score(y_test, line_y))
    print('Variance score of ransac: %.2f' % r2_score(y_test, line_y_ransac))

TypeError: '<' not supported between instances of 'float' and 'str'

In [10]:
def yieldPredict(test_input):
    y_predicted = []
    for m in model_list:
        ys = m[0].predict(test_input)
        y_predicted.append([ys,m[1],m[2]])
        #print(m[1], m[2])
        #print(ys)
    y_predicted.sort(reverse=True)
    for i in y_predicted:
        print(i[0], " ", i[1], " ", i[2])
        
yieldPredict([[100,25]])

[0.79047157]   Jowar   Kharif
[0.62888308]   Other Cereals & Millets   Rabi
[0.62678923]   Maize   Rabi
[0.59788771]   Groundnut   Kharif
[0.55757831]   Cotton(lint)   Kharif
[0.55580617]   Small millets   Kharif
[0.54972917]   Maize   Summer
[0.52658752]   Gram   Rabi
[0.52549181]   Maize   Kharif
[0.52251347]   Pulses total   Kharif
[0.52173526]   Groundnut   Summer
[0.47954149]   Jowar   Rabi
[0.47669421]   Linseed   Rabi
[0.47613254]   Ragi   Kharif
[0.46394131]   Other  Rabi pulses   Rabi
[0.44318006]   Other Kharif pulses   Kharif
[0.44152264]   Safflower   Rabi
[0.43975045]   Total foodgrain   Kharif
[0.43609505]   Sesamum   Rabi
[0.43408658]   Castor seed   Kharif
[0.43402139]   Soyabean   Kharif
[0.43036303]   Moong(Green Gram)   Kharif
[0.41814801]   Sunflower   Rabi
[0.40983239]   Urad   Kharif
[0.40733268]   Rapeseed &Mustard   Rabi
[0.39863912]   Gram   Kharif
[0.39557901]   Small millets   Rabi
[0.38351034]   Cotton(lint)   Whole Year
[0.38234364]   Sugarcane   Kharif
[0.